# Import Library

In [1]:
import pandas as pd, numpy as np, requests, os, json
from bs4 import BeautifulSoup as BS

# Static Data

In [2]:
source = 'https://www.bajajallianz.com/health-insurance-plans/health-insurance-documents.html'
author = 'Bajaj Allianz'

# Scraper

In [3]:
content =  requests.get('https://www.bajajallianz.com/health-insurance-plans/health-insurance-documents.html').content
soup = BS(content, 'lxml')

table_data = soup.find('section', {'class': 'typography-section'}).find('tbody').find_all('tr')


In [4]:
len(table_data)

25

In [5]:
data = {}
columns_names = {
    1:"Brochure", 2:"Customer Information Sheet", 3:"Policy Wordings", 4:"Proposal Form"
}

for row in table_data:
    cols = row.find_all('td') 
    data[cols[0].text.strip()] = {
        columns_names[idx]: [
            link['href']
            for link in col.find_all('a')
        ]
        for idx,col in enumerate(cols[1:],start=1)
    }


# Save data

In [6]:
os.makedirs('../data', exist_ok=True)

with open('../data/bajaj_insurance_documents_statewise.json', 'w') as f:
    json.dump(
        {
            "source": source,
            "author": author,
            "data": data
        }, 
        f, indent=4
    )

# Save PDF FILES

In [1]:
import os
import json
import requests
from pathlib import Path
from tqdm import tqdm

# 🔹 Change this path to where your JSON file is saved
json_file = "../data/bajaj_insurance_documents_statewise.json"

# 🔹 Choose your download directory
download_dir = Path("downloaded_pdfs")
download_dir.mkdir(parents=True, exist_ok=True)

# Load JSON data
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract all PDF links
pdf_links = []
for state, categories in data["data"].items():
    for category, urls in categories.items():
        for url in urls:
            if url.lower().endswith(".pdf"):
                pdf_links.append((state, category, url))

print(f"Found {len(pdf_links)} PDF files to download...")

# Download PDFs
for state, category, url in tqdm(pdf_links):
    try:
        response = requests.get(url, stream=True, timeout=15)
        response.raise_for_status()

        # Create subdirectory for each state
        state_dir = download_dir / state.replace(" ", "_").replace("&", "and")
        state_dir.mkdir(parents=True, exist_ok=True)

        # Save file with a clean name
        filename = f"{category.replace(' ', '_')}_{os.path.basename(url)}"
        filepath = state_dir / filename

        with open(filepath, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"✅ Downloaded: {filepath}")

    except Exception as e:
        print(f"❌ Failed to download {url} - {e}")


Found 174 PDF files to download...


  1%|          | 1/174 [00:09<27:41,  9.60s/it]

✅ Downloaded: downloaded_pdfs\Mee_Kosam-_Andhra_Pradesh_and_Telangana\Brochure_Mee_Kosam-Andhra_Pradesh_Telangana_Brochure.pdf


  1%|          | 2/174 [00:21<31:18, 10.92s/it]

✅ Downloaded: downloaded_pdfs\Mee_Kosam-_Andhra_Pradesh_and_Telangana\Brochure_Mee-Kosam-Andhra-Pradesh-Telangana.pdf


...

✅ Downloaded: downloaded_pdfs\Khyed_kyi_ched_du-_Ladakh\Policy_Wordings_Khyed_kyi_ched_du-Ladakh_Policy_Wordings.pdf


 99%|█████████▉| 173/174 [08:34<00:02,  2.18s/it]

✅ Downloaded: downloaded_pdfs\Khyed_kyi_ched_du-_Ladakh\Proposal_Form_Khyed_kyi_ched_du-Ladakh_Proposal_Form.pdf


100%|██████████| 174/174 [08:35<00:00,  2.96s/it]

✅ Downloaded: downloaded_pdfs\Khyed_kyi_ched_du-_Ladakh\Proposal_Form_Khyed_kyi_ched_du-Ladakh_Digital_PF.pdf
